https://github.com/ahans30/Binoculars

In [1]:
!git clone https://github.com/ahans30/Binoculars.git
%cd Binoculars
!pip install -e .
!pip install datasets

fatal: destination path 'Binoculars' already exists and is not an empty directory.
/content/Binoculars
Obtaining file:///content/Binoculars
  Preparing metadata (setup.py) ... done
  Using cached https://github.com/huggingface/transformers/archive/refs/tags/v4.31.0.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Attempting uninstall: Binoculars
    Found existing installation: Binoculars 0.0.10
    Uninstalling Binoculars-0.0.10:
      Successfully uninstalled Binoculars-0.0.10
  Running setup.py develop for Binoculars


In [2]:
from binoculars import Binoculars

bino = Binoculars()

# ChatGPT (GPT-4) output when prompted with “Can you write a few sentences about a capybara that is an astrophysicist?"
sample_string = '''Dr. Capy Cosmos, a capybara unlike any other, astounded the scientific community with his
groundbreaking research in astrophysics. With his keen sense of observation and unparalleled ability to interpret
cosmic data, he uncovered new insights into the mysteries of black holes and the origins of the universe. As he
peered through telescopes with his large, round eyes, fellow researchers often remarked that it seemed as if the
stars themselves whispered their secrets directly to him. Dr. Cosmos not only became a beacon of inspiration to
aspiring scientists but also proved that intellect and innovation can be found in the most unexpected of creatures.'''

print(bino.compute_score(sample_string))  # 0.75661373
print(bino.predict(sample_string))  # 'Most likely AI-Generated'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using pad_token, but it is not set yet.


0.771276593208313
Most likely AI-generated


In [3]:
temp = "ok is this working"
print(bino.compute_score(temp))  # 1.1117020845413208
print(bino.predict(temp))  # Most likely human-generated

1.1117020845413208
Most likely human-generated


In [ ]:
import os
from datasets import load_dataset, load_from_disk

SAVE_PATH = '/content/data/wiki_dataset'

def get_wikipedia_dataset():
    # Check if dataset already exists on disk
    if os.path.exists(SAVE_PATH):
        print("Loading dataset from disk...")
        dataset = load_from_disk(SAVE_PATH)
    else:
        print("Dataset not found on disk. Downloading from Hugging Face...")
        # Create directory if it doesn't exist
        os.makedirs('/content/data', exist_ok=True)

        # Download and load the dataset
        dataset = load_dataset("wikipedia", "20220301.en")

        # Save to disk
        print("Saving dataset to disk...")
        dataset.save_to_disk(SAVE_PATH)

    print("Dataset is ready!")
    return dataset

In [ ]:
legacy_dataset = get_wikipedia_dataset()


Loading dataset from disk...


Loading dataset from disk:   0%|          | 0/41 [00:00<?, ?it/s]

Dataset is ready!


In [ ]:
import json
from datasets import Dataset, DatasetDict

with open("/content/scraped_wiki_articles_2_2025.json", "r", encoding="utf-8") as file:
    data = json.load(file)

for entry in data:
    entry["text"] = entry.pop("content")

dataset = Dataset.from_list(data)

new_articles_dataset = DatasetDict({"train": dataset})

print(new_articles_dataset)
print(legacy_dataset)


DatasetDict({
    train: Dataset({
        features: ['title', 'url', 'text'],
        num_rows: 1990
    })
})
DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 6458670
    })
})


In [ ]:
with open("/content/scraped_wiki_articles_8_2023.json", "r", encoding="utf-8") as file:
    data = json.load(file)

for entry in data:
    entry["text"] = entry.pop("content")

dataset = Dataset.from_list(data)

august_2023_dataset = DatasetDict({"train": dataset})
print(august_2023_dataset)

DatasetDict({
    train: Dataset({
        features: ['title', 'url', 'text'],
        num_rows: 1000
    })
})


In [9]:
import json
import re

all_scores = []
non_ai = 0
ai = 0
def ai_score(lines, truncate_to=5000):
    print(f"number of lines: {len(lines)}")
    global non_ai, ai
    scores = []
    for ind, line in enumerate(lines):
        print(len(line))
        score = bino.compute_score(' '.join(line.strip().split()[:truncate_to]))
        print(score, ind)
        scores.append((score, ind))

        prediction = bino.predict(' '.join(line.strip().split()[:truncate_to]))
        print(prediction)
        if "AI" in prediction:
            ai += 1
        else:
            non_ai += 1

    return scores

def clean_text(text):
    # Replace all newlines with a unique string
    text = text.replace('\n', '!@#').replace('\t', ' ')
    # Split the text by the unique string
    split_text = text.split('!@#')
    # Remove elements with 5 or fewer words
    split_text = [segment for segment in split_text if len(segment.split()) > 5]
    # Rejoin the text
    text = ' '.join(split_text)
    # Remove "References" and "External links" sections
    text = re.sub(r'== References ==.*|== External links ==.*', '', text, flags=re.DOTALL)

    return text.strip()

def extract_title_text_and_url_legacy(dataset):
    titles, texts, urls = [], [], []

    for i in range(850):

      # common crawl has empty title and url
      title = dataset["train"][i]["title"]
      content = dataset["train"][i]["text"]
      url = dataset["train"][i]["url"]
      cleaned_text = clean_text(content)

      if title and cleaned_text:
          titles.append(title)
          texts.append(cleaned_text)
          urls.append(url)

    return titles, texts, urls

def process_files(dataset):
    titles, texts, urls = extract_title_text_and_url_legacy(dataset)
    texts = [' '.join(text.strip().split()[:400]) for text in texts]
    # final_texts, final_titles, final_urls = [],[],[]
    # for ind, text in enumerate(texts):
    #     if len(text.split()) >= 100:
    #         final_texts.append(texts[ind])
    #         final_titles.append(titles[ind])
    #         final_urls.append(urls[ind])

    print(f"number of texts: {len(texts)}")

    bino_scores = ai_score(texts)
    all_scores.append(bino_scores)

    # with open(output_file, 'w', encoding='utf-8') as outfile:
    #     for title, score, url, text in zip(final_titles, bino_scores, final_urls, final_texts):
    #         outfile.write(f"{title}, {score[0]}\n")


In [ ]:
output_file = "/content/legacy_wikipedia.json"
input_file = ""

**Legacy Articles**

In [ ]:
process_files(legacy_dataset)

number of texts: 1200
number of lines: 1200
2645
0.9932432174682617 0
Most likely human-generated
2767
0.9781022071838379 1
Most likely human-generated
2493
1.0068492889404297 2
Most likely human-generated
2338
0.916167676448822 3
Most likely human-generated
2451
0.9539749026298523 4
Most likely human-generated
2296
0.9215686321258545 5
Most likely human-generated
2637
0.9357143044471741 6
Most likely human-generated
2602
1.0349650382995605 7
Most likely human-generated
2385
1.0227272510528564 8
Most likely human-generated
847
0.9418604373931885 9
Most likely human-generated
2412
0.9491525292396545 10
Most likely human-generated
2159
1.0 11
Most likely human-generated
2424
0.9235293865203857 12
Most likely human-generated
2420
0.9364162087440491 13
Most likely human-generated
2555
0.8994082808494568 14
Most likely human-generated
2643
0.9444444179534912 15
Most likely human-generated
2535
0.9296296238899231 16
Most likely human-generated
2466
0.9934210777282715 17
Most likely human-gen

In [ ]:
print(all_scores)
print(ai)
print(non_ai)

[[(0.9932432174682617, 0), (0.9781022071838379, 1), (1.0068492889404297, 2), (0.916167676448822, 3), (0.9539749026298523, 4), (0.9215686321258545, 5), (0.9357143044471741, 6), (1.0349650382995605, 7), (1.0227272510528564, 8), (0.9418604373931885, 9), (0.9491525292396545, 10), (1.0, 11), (0.9235293865203857, 12), (0.9364162087440491, 13), (0.8994082808494568, 14), (0.9444444179534912, 15), (0.9296296238899231, 16), (0.9934210777282715, 17), (0.9099099040031433, 18), (0.9934210777282715, 19), (0.9937106966972351, 20), (0.9473684430122375, 21), (0.9810126423835754, 22), (0.9485294222831726, 23), (1.0214285850524902, 24), (0.9726027250289917, 25), (0.9025641083717346, 26), (0.9714285731315613, 27), (0.9411764740943909, 28), (0.8644067645072937, 29), (0.987261176109314, 30), (1.0, 31), (0.8969465494155884, 32), (0.8531468510627747, 33), (0.9798387289047241, 34), (0.9520547986030579, 35), (0.9932432174682617, 36), (0.9430894255638123, 37), (0.9874213933944702, 38), (0.984375, 39), (0.9764705

In [ ]:
import numpy as np

score_nums = []

for score, _ in all_scores[0]:
  # print(score)
  score_nums.append(score)

mean = np.mean(score_nums)
median = np.median(score_nums)
std = np.std(score_nums)

print(f"Mean: {mean}")
print(f"Median: {median}")
print(f"Std dev: {std}")




Mean: 0.9742311434447766
Median: 0.9726027250289917
Std dev: 0.04382572666444521


**New Articles**

In [ ]:
process_files(new_articles_dataset)
print(all_scores)
print(ai)
print(non_ai)

number of texts: 1200
number of lines: 1200
753
0.9219858050346375 0
Most likely human-generated
2671
0.9943181872367859 1
Most likely human-generated
947
0.8813559412956238 2
Most likely human-generated
1221
1.045454502105713 3
Most likely human-generated
232
1.142045497894287 4
Most likely human-generated
327
0.9610389471054077 5
Most likely human-generated
557
1.0561797618865967 6
Most likely human-generated
929
1.1483516693115234 7
Most likely human-generated
2306
0.9873417615890503 8
Most likely human-generated
1028
0.9935483932495117 9
Most likely human-generated
2357
1.0616438388824463 10
Most likely human-generated
1691
0.9803921580314636 11
Most likely human-generated
1272
1.0338983535766602 12
Most likely human-generated
652
1.029197096824646 13
Most likely human-generated
1390
0.9857142567634583 14
Most likely human-generated
254
0.9290322661399841 15
Most likely human-generated
1012
0.9772727489471436 16
Most likely human-generated
2592
0.9122137427330017 17
Most likely hum

In [ ]:
score_nums = []

for score, _ in all_scores[0]:
  # print(score)
  score_nums.append(score)

mean = np.mean(score_nums)
median = np.median(score_nums)
std = np.std(score_nums)

print(f"Mean: {mean}")
print(f"Median: {median}")
print(f"Std dev: {std}")

Mean: 0.9804497489829859
Median: 0.9795918464660645
Std dev: 0.07997871609350253


August 2023 Articles

In [ ]:
process_files(august_2023_dataset)

number of texts: 1000
number of lines: 1000
2345
0.8902438879013062 0
Most likely human-generated
1275
1.0512820482254028 1
Most likely human-generated
949
0.9200000166893005 2
Most likely human-generated
2510
0.9753086566925049 3
Most likely human-generated
2510
0.9753086566925049 4
Most likely human-generated
2481
0.9668874144554138 5
Most likely human-generated
2330
1.0125786066055298 6
Most likely human-generated
2509
0.9357143044471741 7
Most likely human-generated
2332
0.9469696879386902 8
Most likely human-generated
638
1.0 9
Most likely human-generated
2779
0.9399999976158142 10
Most likely human-generated
812
0.9347826242446899 11
Most likely human-generated
2481
0.9668874144554138 12
Most likely human-generated
1276
1.0 13
Most likely human-generated
2413
0.9852941036224365 14
Most likely human-generated
2481
0.9668874144554138 15
Most likely human-generated
817
0.9754601120948792 16
Most likely human-generated
2467
1.0758620500564575 17
Most likely human-generated
2451
0.984

In [ ]:
print(all_scores)
print(ai)
print(non_ai)

[[(0.8902438879013062, 0), (1.0512820482254028, 1), (0.9200000166893005, 2), (0.9753086566925049, 3), (0.9753086566925049, 4), (0.9668874144554138, 5), (1.0125786066055298, 6), (0.9357143044471741, 7), (0.9469696879386902, 8), (1.0, 9), (0.9399999976158142, 10), (0.9347826242446899, 11), (0.9668874144554138, 12), (1.0, 13), (0.9852941036224365, 14), (0.9668874144554138, 15), (0.9754601120948792, 16), (1.0758620500564575, 17), (0.9846153855323792, 18), (1.0458015203475952, 19), (1.0458015203475952, 20), (0.9632353186607361, 21), (0.9597315192222595, 22), (0.9597315192222595, 23), (0.9597315192222595, 24), (0.9597315192222595, 25), (0.9597315192222595, 26), (0.9597315192222595, 27), (0.9597315192222595, 28), (1.0458015203475952, 29), (1.0458015203475952, 30), (1.0458015203475952, 31), (1.0458015203475952, 32), (1.0458015203475952, 33), (1.0458015203475952, 34), (0.988095223903656, 35), (0.988095223903656, 36), (1.0458015203475952, 37), (1.0, 38), (1.0458015203475952, 39), (1.045801520347

August 2024 Articles

In [6]:
import json
from datasets import Dataset, DatasetDict

with open("/content/scraped_wiki_articles_8_2024.json", "r", encoding="utf-8") as file:
    data = json.load(file)

for entry in data:
    entry["text"] = entry.pop("content")

dataset = Dataset.from_list(data)

august_2024_dataset = DatasetDict({"train": dataset})
print(august_2024_dataset)

DatasetDict({
    train: Dataset({
        features: ['title', 'url', 'text'],
        num_rows: 850
    })
})


In [10]:
process_files(august_2024_dataset)

number of texts: 850
number of lines: 850
2408
1.0125000476837158 0
Most likely human-generated
2377
0.9477124214172363 1
Most likely human-generated
435
1.0232558250427246 2
Most likely human-generated
2543
0.9387755393981934 3
Most likely human-generated
1177
0.902953565120697 4
Most likely human-generated
2444
0.9870967864990234 5
Most likely human-generated
1071
0.9441340565681458 6
Most likely human-generated
2383
0.9919678568840027 7
Most likely human-generated
2486
0.9851852059364319 8
Most likely human-generated
3113
0.9371428489685059 9
Most likely human-generated
3113
0.9371428489685059 10
Most likely human-generated
1166
1.0413793325424194 11
Most likely human-generated
2297
1.0 12
Most likely human-generated
1083
0.948387086391449 13
Most likely human-generated
1083
0.948387086391449 14
Most likely human-generated
2305
1.0188679695129395 15
Most likely human-generated
2564
1.0 16
Most likely human-generated
2579
0.9820359349250793 17
Most likely human-generated
2526
1.02499

In [11]:
print(all_scores)
print(ai)
print(non_ai)

[[(1.0125000476837158, 0), (0.9477124214172363, 1), (1.0232558250427246, 2), (0.9387755393981934, 3), (0.902953565120697, 4), (0.9870967864990234, 5), (0.9441340565681458, 6), (0.9919678568840027, 7), (0.9851852059364319, 8), (0.9371428489685059, 9), (0.9371428489685059, 10), (1.0413793325424194, 11), (1.0, 12), (0.948387086391449, 13), (0.948387086391449, 14), (1.0188679695129395, 15), (1.0, 16), (0.9820359349250793, 17), (1.024999976158142, 18), (1.024999976158142, 19), (1.0129870176315308, 20), (0.9819276928901672, 21), (0.9880239367485046, 22), (1.019736886024475, 23), (0.9370370507240295, 24), (0.9370370507240295, 25), (0.9370370507240295, 26), (0.9370370507240295, 27), (0.9370370507240295, 28), (0.9936708807945251, 29), (0.9041666388511658, 30), (0.9192546606063843, 31), (0.9618320465087891, 32), (0.9618320465087891, 33), (0.9618320465087891, 34), (0.9387755393981934, 35), (0.9720930457115173, 36), (0.8395061492919922, 37), (0.8612244725227356, 38), (0.814393937587738, 39), (0.97

Common Crawl May 2022

In [ ]:
import json
from datasets import Dataset, DatasetDict

with open("/content/CC-MAIN-2022-05.json", "r", encoding="utf-8") as file:
    data = json.load(file)

sites_data = [{"text": site.pop("text"), "title": "temp", "url": ""} for site in data["sites"]]

dataset = Dataset.from_list(sites_data)

cc_2022_dict = DatasetDict({"train": dataset})

print(cc_2022_dict)

DatasetDict({
    train: Dataset({
        features: ['text', 'title', 'url'],
        num_rows: 1200
    })
})


In [ ]:
process_files(cc_2022_dict)

number of texts: 889
number of lines: 889
50
0.6132075190544128 0
Most likely AI-generated
745
0.9299362897872925 1
Most likely human-generated
1647
0.9878787994384766 2
Most likely human-generated
152
1.0585365295410156 3
Most likely human-generated
2302
0.9958158731460571 4
Most likely human-generated
5597
1.006622552871704 5
Most likely human-generated
137
1.0382165908813477 6
Most likely human-generated
1403
1.01630437374115 7
Most likely human-generated
1277
1.0703517198562622 8
Most likely human-generated
922
0.9730769395828247 9
Most likely human-generated
3108
0.9534883499145508 10
Most likely human-generated
2465
0.9322034120559692 11
Most likely human-generated
2304
0.9589743614196777 12
Most likely human-generated
2600
0.9951456189155579 13
Most likely human-generated
2854
0.9890710115432739 14
Most likely human-generated
1098
0.9720670580863953 15
Most likely human-generated
864
0.9528301954269409 16
Most likely human-generated
29
1.3684210777282715 17
Most likely human-gen

In [ ]:
print(all_scores)
print(ai)
print(non_ai)

[[], [(0.6132075190544128, 0), (0.9299362897872925, 1), (0.9878787994384766, 2), (1.0585365295410156, 3), (0.9958158731460571, 4), (1.006622552871704, 5), (1.0382165908813477, 6), (1.01630437374115, 7), (1.0703517198562622, 8), (0.9730769395828247, 9), (0.9534883499145508, 10), (0.9322034120559692, 11), (0.9589743614196777, 12), (0.9951456189155579, 13), (0.9890710115432739, 14), (0.9720670580863953, 15), (0.9528301954269409, 16), (1.3684210777282715, 17), (1.0042372941970825, 18), (0.9621621370315552, 19), (0.9807692170143127, 20), (0.9375, 21), (1.01578950881958, 22), (0.9596412777900696, 23), (1.046632170677185, 24), (0.8783068656921387, 25), (0.9596412777900696, 26), (1.1814346313476562, 27), (1.0360825061798096, 28), (0.9484536051750183, 29), (0.9487179517745972, 30), (0.9418604373931885, 31), (1.0789474248886108, 32), (0.9656862616539001, 33), (0.9742268323898315, 34), (0.9716312289237976, 35), (0.9477124214172363, 36), (0.9882352948188782, 37), (1.2333333492279053, 38), (0.97692

Common Crawl March 2023

In [ ]:
import json
from datasets import Dataset, DatasetDict

with open("/content/CC-MAIN-2023-40.json", "r", encoding="utf-8") as file:
    data = json.load(file)

sites_data = [{"text": site.pop("text"), "title": "temp", "url": ""} for site in data["sites"]]

dataset = Dataset.from_list(sites_data)

cc_2023_dict = DatasetDict({"train": dataset})

print(cc_2023_dict)

DatasetDict({
    train: Dataset({
        features: ['text', 'title', 'url'],
        num_rows: 1200
    })
})


In [ ]:
process_files(cc_2023_dict)

number of texts: 875
number of lines: 875
31
0.847161591053009 0
Most likely AI-generated
86
1.0152671337127686 1
Most likely human-generated
72
1.0139859914779663 2
Most likely human-generated
316
0.9504950642585754 3
Most likely human-generated
2058
0.9893048405647278 4
Most likely human-generated
279
1.0202020406723022 5
Most likely human-generated
318
0.9704433679580688 6
Most likely human-generated
1757
0.9731543660163879 7
Most likely human-generated
1375
0.9019607901573181 8
Most likely human-generated
2132
0.8977272510528564 9
Most likely human-generated
144
0.9679999947547913 10
Most likely human-generated
262
0.9482758641242981 11
Most likely human-generated
2730
0.9593023061752319 12
Most likely human-generated
2683
0.9409282803535461 13
Most likely human-generated
132
1.1526718139648438 14
Most likely human-generated
2095
1.0068026781082153 15
Most likely human-generated
688
0.9433962106704712 16
Most likely human-generated
1288
0.9847328066825867 17
Most likely human-gener

In [ ]:
print(all_scores)
print(ai)
print(non_ai)

[[(0.847161591053009, 0), (1.0152671337127686, 1), (1.0139859914779663, 2), (0.9504950642585754, 3), (0.9893048405647278, 4), (1.0202020406723022, 5), (0.9704433679580688, 6), (0.9731543660163879, 7), (0.9019607901573181, 8), (0.8977272510528564, 9), (0.9679999947547913, 10), (0.9482758641242981, 11), (0.9593023061752319, 12), (0.9409282803535461, 13), (1.1526718139648438, 14), (1.0068026781082153, 15), (0.9433962106704712, 16), (0.9847328066825867, 17), (0.9263157844543457, 18), (1.0084388256072998, 19), (0.9340101480484009, 20), (0.8457711338996887, 21), (0.991304337978363, 22), (1.1408450603485107, 23), (0.9817073345184326, 24), (0.9755101799964905, 25), (1.0127660036087036, 26), (1.051162838935852, 27), (1.0046948194503784, 28), (1.0, 29), (1.3333333730697632, 30), (0.9567099809646606, 31), (1.0446927547454834, 32), (0.9854369163513184, 33), (0.8926829099655151, 34), (0.9659863710403442, 35), (0.9436619877815247, 36), (0.9793814420700073, 37), (0.9441860318183899, 38), (0.982658982

Common Crawl March 2024

In [ ]:
import json
from datasets import Dataset, DatasetDict

with open("/content/CC-MAIN-2024-33.json", "r", encoding="utf-8") as file:
    data = json.load(file)

sites_data = [{"text": site.pop("text"), "title": "temp", "url": ""} for site in data["sites"]]

dataset = Dataset.from_list(sites_data)

cc_2024_dict = DatasetDict({"train": dataset})

print(cc_2024_dict)

DatasetDict({
    train: Dataset({
        features: ['text', 'title', 'url'],
        num_rows: 1200
    })
})


In [ ]:
process_files(cc_2024_dict)

number of texts: 852
number of lines: 852
43
1.1944444179534912 0
Most likely human-generated
163
0.93034827709198 1
Most likely human-generated
41
0.9803921580314636 2
Most likely human-generated
1233
0.9568345546722412 3
Most likely human-generated
301
0.9950248599052429 4
Most likely human-generated
65
0.9618644118309021 5
Most likely human-generated
2298
0.9947916865348816 6
Most likely human-generated
51
0.9208633303642273 7
Most likely human-generated
696
1.0176211595535278 8
Most likely human-generated
220
0.9668246507644653 9
Most likely human-generated
51
0.9208633303642273 10
Most likely human-generated
406
1.0 11
Most likely human-generated
850
1.0133333206176758 12
Most likely human-generated
66
0.9657142758369446 13
Most likely human-generated
203
0.9562841653823853 14
Most likely human-generated
1214
0.9434782862663269 15
Most likely human-generated
157
1.2333333492279053 16
Most likely human-generated
1768
0.95333331823349 17
Most likely human-generated
3481
1.0264550447

In [ ]:
print(all_scores)
print(ai)
print(non_ai)

[[(1.1944444179534912, 0), (0.93034827709198, 1), (0.9803921580314636, 2), (0.9568345546722412, 3), (0.9950248599052429, 4), (0.9618644118309021, 5), (0.9947916865348816, 6), (0.9208633303642273, 7), (1.0176211595535278, 8), (0.9668246507644653, 9), (0.9208633303642273, 10), (1.0, 11), (1.0133333206176758, 12), (0.9657142758369446, 13), (0.9562841653823853, 14), (0.9434782862663269, 15), (1.2333333492279053, 16), (0.95333331823349, 17), (1.026455044746399, 18), (1.038095235824585, 19), (1.038095235824585, 20), (1.0, 21), (1.1136363744735718, 22), (0.9811320900917053, 23), (0.9372385144233704, 24), (0.8306451439857483, 25), (0.8933333158493042, 26), (0.981249988079071, 27), (1.0496894121170044, 28), (0.9775280952453613, 29), (1.0204081535339355, 30), (0.954285740852356, 31), (0.995192289352417, 32), (0.9459459185600281, 33), (0.9895833134651184, 34), (1.0055866241455078, 35), (1.0273224115371704, 36), (1.0101523399353027, 37), (0.9869281053543091, 38), (1.1678831577301025, 39), (1.01092

Common Crawl 2025

In [ ]:
import json
from datasets import Dataset, DatasetDict

with open("/content/CC-MAIN-2025-08.json", "r", encoding="utf-8") as file:
    data = json.load(file)

sites_data = [{"text": site.pop("text"), "title": "temp", "url": ""} for site in data["sites"]]

dataset = Dataset.from_list(sites_data)

cc_2025_dict = DatasetDict({"train": dataset})

print(cc_2025_dict)

DatasetDict({
    train: Dataset({
        features: ['text', 'title', 'url'],
        num_rows: 1200
    })
})


In [ ]:
process_files(cc_2025_dict)

number of texts: 875
number of lines: 875
385
0.969072163105011 0
Most likely human-generated
144
0.8521126508712769 1
Most likely AI-generated
17
0.7348484992980957 2
Most likely AI-generated
1222
1.0086207389831543 3
Most likely human-generated
124
1.115384578704834 4
Most likely human-generated
2655
0.9494949579238892 5
Most likely human-generated
783
1.0346534252166748 6
Most likely human-generated
837
0.9753086566925049 7
Most likely human-generated
679
1.0107526779174805 8
Most likely human-generated
51
1.2767295837402344 9
Most likely human-generated
202
0.9627659320831299 10
Most likely human-generated
1684
0.956204354763031 11
Most likely human-generated
962
1.026455044746399 12
Most likely human-generated
1055
1.0191388130187988 13
Most likely human-generated
1203
0.9470198750495911 14
Most likely human-generated
2459
0.9464285969734192 15
Most likely human-generated
194
1.0117647647857666 16
Most likely human-generated
1067
0.9254385828971863 17
Most likely human-generated
2

In [ ]:
print(all_scores)
print(ai)
print(non_ai)

[[(0.969072163105011, 0), (0.8521126508712769, 1), (0.7348484992980957, 2), (1.0086207389831543, 3), (1.115384578704834, 4), (0.9494949579238892, 5), (1.0346534252166748, 6), (0.9753086566925049, 7), (1.0107526779174805, 8), (1.2767295837402344, 9), (0.9627659320831299, 10), (0.956204354763031, 11), (1.026455044746399, 12), (1.0191388130187988, 13), (0.9470198750495911, 14), (0.9464285969734192, 15), (1.0117647647857666, 16), (0.9254385828971863, 17), (1.0358566045761108, 18), (0.9954128265380859, 19), (1.0228310823440552, 20), (0.9890710115432739, 21), (0.9521530866622925, 22), (0.9476439952850342, 23), (0.9932432174682617, 24), (1.063218355178833, 25), (1.0858895778656006, 26), (0.9139785170555115, 27), (1.0, 28), (1.0199999809265137, 29), (0.993630588054657, 30), (1.2307692766189575, 31), (0.8035714030265808, 32), (1.0248756408691406, 33), (0.7189781069755554, 34), (0.9707112908363342, 35), (0.9375, 36), (0.9375, 37), (1.1794872283935547, 38), (1.196969747543335, 39), (0.83333331346